In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns

from postprocessor.core.processes.findpeaks import findpeaks

In [ ]:
from src.utils import simple_median_plot
from src.crosscorr import crosscorr
from src.synthetic import sinusoid, fitzhugh_nagumo, gillespie_noise

# Get std dev of simulated oscillators

In [ ]:
std_sinusoid = np.sqrt(2)/2 #theoretical

In [ ]:
fhn = fitzhugh_nagumo(timeaxis=np.linspace(0,500,500), ext_stimulus=0.4, tau=12.5, a=0.7, b=0.82)

In [ ]:
std_fhn = np.std(fhn)

# Load data

Load ACF

In [ ]:
filepath = "../data/raw/is26643_htb2mCherry_flavin_acf.csv"
autocorr_df = pd.read_csv(filepath, index_col=[0,1,2])
autocorr_df = autocorr_df.dropna()

In [ ]:
autocorr_df

Load time series

In [ ]:
timeseries_df = pd.read_csv("../data/raw/is26643_htb2mCherry_flavin_timeseries.csv", index_col=[0,1,2])
#timeseries_df = pd.read_csv("../data/raw/is26643_htb2mCherry_mCherry_timeseries.csv", index_col=[0,1,2])
#timeseries_df = pd.read_csv("../data/raw/is31594_fy4_timeseries.csv", index_col=[0,1,2])

timeseries_df = timeseries_df.dropna()

In [ ]:
mean_std = timeseries_df.std().mean()

In [ ]:
# Scale so std dev is equal to std dev of sinusoid
timeseries_df *= (std_sinusoid)/(mean_std)

In [ ]:
autocorr_df = crosscorr.as_function(
    timeseries_df, stationary=False, normalised=True, only_pos=True
)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(timeseries_df.iloc[0])
ax.xaxis.set_major_locator(ticker.MultipleLocator(20))
ax.set_xlabel("Time point")
ax.set_ylabel("Signal")

# Draw ACF

In [ ]:
fig, ax = plt.subplots()

# draw acf
simple_median_plot(
    autocorr_df,
    xlabel="Lag (time points)",
    ylabel="Autocorrelation function",
    ax=ax,
)
# and axes
plt.axhline(0, color="k")
plt.axvline(0, color="k")

# Find peaks, troughs

In [ ]:
# find peaks & troughs
mean_acf_df = autocorr_df.mean().to_frame().T
peaks_df = findpeaks.as_function(mean_acf_df)
troughs_df = findpeaks.as_function(-mean_acf_df)

# datatype conversions
timeaxis = mean_acf_df.columns.to_numpy()
timeaxis = timeaxis.astype(float)
mean_acf = mean_acf_df.to_numpy()[0]
peaks_mask = peaks_df.to_numpy()[0] != 0
troughs_mask = troughs_df.to_numpy()[0] != 0

Draw locations of peaks & troughs

In [ ]:
# draw where peaks are
fig, ax = plt.subplots()
ax.plot(timeaxis, mean_acf)
ax.scatter(timeaxis[peaks_mask], mean_acf[peaks_mask])
ax.scatter(timeaxis[troughs_mask], mean_acf[troughs_mask])
ax.xaxis.set_major_locator(ticker.MultipleLocator(20))
ax.set_xlabel("Time point")
ax.set_ylabel("ACF")

# Estimate period

In [ ]:
# Get location of first peak, as an estimate of period
est_period = timeaxis[peaks_mask][0]
print(est_period)

# Fit exponential

Define functions (TODO: move to src)

In [ ]:
def model_func(t, K, C):
    return (1 - C) * np.exp(-K * t) + C


def fit_exp_nonlinear(t, y, p0):
    opt_parms, parm_cov = sp.optimize.curve_fit(model_func, t, y, p0, maxfev=1000)
    K, C = opt_parms
    return K, C


def fit_mean(
    array,
    initial_K,
    initial_C=0,
):
    # get mean time series
    mean_df = array.mean().to_frame().T
    timeaxis = mean_df.columns.to_numpy()
    mean_acf = mean_df.to_numpy()[0]

    # initial guess is the decay function in acf plot
    initial_guess = [initial_K, initial_C]

    # fit mean
    est_coeffs = fit_exp_nonlinear(
        timeaxis,
        mean_acf,
        p0=initial_guess,
    )

    return est_coeffs


def fit_peak_trough(
    array,
    initial_K,
    initial_C=0,
):
    """
    array: 2d numpy array
    """
    # find peaks & troughs
    mean_df = array.mean().to_frame().T
    peaks_df = findpeaks.as_function(mean_df)
    troughs_df = findpeaks.as_function(-mean_df)
    # datatype conversions
    timeaxis = mean_df.columns.to_numpy()
    mean_acf = mean_df.to_numpy()[0]
    peaks_mask = peaks_df.to_numpy()[0] != 0
    troughs_mask = troughs_df.to_numpy()[0] != 0
    # add (0,1) to datapoints
    peaks_mask[0] = True
    troughs_mask[0] = True

    # initial guess is the decay function in acf plot
    initial_guess = [initial_K, initial_C]

    # fit peaks
    upper_coeffs = fit_exp_nonlinear(
        timeaxis[peaks_mask],
        mean_acf[peaks_mask],
        p0=initial_guess,
    )
    # fit troughs
    lower_coeffs = fit_exp_nonlinear(
        timeaxis[troughs_mask],
        mean_acf[troughs_mask],
        p0=initial_guess,
    )

    return upper_coeffs, lower_coeffs

Scale lag axis to make it in units of periods.

In [ ]:
scaling_factor = 1 / est_period

In [ ]:
timeaxis_scaled = timeaxis * scaling_factor

In [ ]:
autocorr_scaled = autocorr_df.copy()
autocorr_scaled.columns = timeaxis_scaled

Fit, with initial guess

In [ ]:
initial_K = 0.05
upper_coeffs, lower_coeffs = fit_peak_trough(autocorr_scaled, initial_K=initial_K)
est_coeffs = fit_mean(autocorr_scaled, initial_K=initial_K)

In [ ]:
print(upper_coeffs)
print(lower_coeffs)
print(est_coeffs)

Draw

In [ ]:
upper_K, upper_C = upper_coeffs
lower_K, lower_C = lower_coeffs
est_K, est_C = est_coeffs

upper_A = 1 - upper_C
lower_A = 1 - lower_C
est_A = 1 - est_C

upper_func = model_func(timeaxis_scaled, upper_coeffs[0], upper_coeffs[1])
lower_func = model_func(timeaxis_scaled, lower_coeffs[0], lower_coeffs[1])
est_func = model_func(timeaxis_scaled, est_coeffs[0], est_coeffs[1])

In [ ]:
plt.plot(timeaxis_scaled, mean_acf)
plt.scatter(timeaxis_scaled[peaks_mask], mean_acf[peaks_mask])
plt.scatter(timeaxis_scaled[troughs_mask], mean_acf[troughs_mask])
plt.plot(timeaxis_scaled, est_func, label="fit to mean ACF")
plt.plot(timeaxis_scaled, upper_func, label="fit to peaks")
plt.plot(timeaxis_scaled, lower_func, label="fit to troughs")
plt.legend()
plt.xlabel("Lag (in periods)")
plt.ylabel("Autocorrelation function")

print(f"upper envelope: {upper_A:.4f} * exp(- {upper_K:.4f}) + {upper_C:.4f}")
print(f"central: {est_A:.4f} * exp(- {est_K:.4f}) + {est_C:.4f}")
print(f"lower envelope: {lower_A:.4f} * exp(- {lower_K:.4f}) + {lower_C:.4f}")

# Decay rate/Death rate

In [ ]:
est_K

Load stats

In [ ]:
deathrate_vs_decay = pd.read_csv("../data/interim/stat/sinusoid/deathrate_vs_decay.csv")
#deathrate_vs_decay = pd.read_csv("../data/interim/stat/fitzhughnagumo/deathrate_vs_decay.csv")

Linear fit

(here with extrapolation -- best practise is to generate Gillespie noise with the extreme deathrate found from real data)

In [ ]:
# Linear
lin_coeffs = np.polyfit(deathrate_vs_decay.deathrate[:5], deathrate_vs_decay.D_central[:5], deg=1)
bestfit_x = np.linspace(0, 0.45, 100)
bestfit_y = lin_coeffs[1] + lin_coeffs[0] * bestfit_x

In [ ]:
# Alternatively, force through origin
slope, _, _, _ = np.linalg.lstsq(deathrate_vs_decay.deathrate[:,np.newaxis], deathrate_vs_decay.D_central)
bestfit_x = np.linspace(0, 0.45, 100)
bestfit_y = slope[0] * bestfit_x

In [ ]:
fig, ax = plt.subplots()
ax.scatter(deathrate_vs_decay.deathrate, deathrate_vs_decay.D_central)
ax.plot(bestfit_x, bestfit_y)
#ax.set_xlim(0,0.06)
#ax.set_ylim(0,0.1)

Estimate death rate

In [ ]:
est_d0 = (est_K - lin_coeffs[1])/lin_coeffs[0]
print(est_d0)

In [ ]:
est_noise_timescale = 1/est_d0
print(est_noise_timescale)

# y-displacement/Birth rate

In [ ]:
upper_C

Load stats

In [ ]:
birthrate_vs_ydispl = pd.read_csv("../data/interim/stat/sinusoid/birthrate_vs_ydispl.csv")
#birthrate_vs_ydispl = pd.read_csv("../data/interim/stat/fitzhughnagumo/birthrate_vs_ydispl.csv")

In [ ]:
birthrate_vs_ydispl

Exponential fit

In [ ]:
B = 0.039172
b_x = birthrate_vs_ydispl.noise_amp
b_y = birthrate_vs_ydispl.C_upper - B

In [ ]:
# Linear
exp_coeffs = np.polyfit(b_x[1:11], np.log(b_y)[1:11], deg=1)
bestfit_x = np.linspace(1, 300, 100)
bestfit_y = exp_coeffs[1] + exp_coeffs[0] * bestfit_x

In [ ]:
fig, ax = plt.subplots()
ax.scatter(b_x, np.log(b_y))
ax.plot(bestfit_x, bestfit_y)

In [ ]:
fig, ax = plt.subplots()
bestfit_exp = np.exp(exp_coeffs[1]) * np.exp(exp_coeffs[0] * bestfit_x)
ax.scatter(b_x, b_y)
ax.plot(bestfit_x, bestfit_exp)

Estimate birth rate

In [ ]:
est_noise_amp = (np.log(upper_C) - exp_coeffs[1])/exp_coeffs[0]
print(est_noise_amp)

In [ ]:
est_k0 = est_noise_amp * est_d0
print(est_k0)

# Simulate with estimated noise parameters

In [ ]:
timeaxis_sim = np.linspace(0, 283, 284)

In [ ]:
signal = sinusoid(timeaxis_sim, amp=np.sqrt(2), freq=1/19, phase=0)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(signal)

In [ ]:
noise = gillespie_noise(
    num_timeseries=1,
    num_timepoints=284,
    noise_amp=est_noise_amp,
    noise_timescale=est_noise_timescale,
    time_final=7500,
    grid_num_intervals=5000,
)

In [ ]:
noise = gillespie_noise(
    num_timeseries=1,
    num_timepoints=500,
    noise_amp=100,
    noise_timescale=20,
    time_final=7500,
    grid_num_intervals=5000,
)

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(noise[0])

In [ ]:
sim = 3 * signal + noise[0]

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(sim)